# Temperature Vegetation Dryness Index (TVDI) Calculation
This notebook calculates the TVDI based on NDVI and LST data using a polynomial fit for dry and wet edges. TVDI is used to monitor drought conditions by comparing the land surface temperature (LST) with the normalized difference vegetation index (NDVI). Below, we will walk through the code required for this calculation step by step.

## Required Libraries
We start by importing the necessary libraries for geospatial data handling, computation, and visualization.


In [ ]:
# Import necessary libraries
try:
    from osgeo import gdal, gdalconst
except ImportError:
    import gdal
    from gdal import gdalconst

import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from datetime import datetime


## Helper Functions

### 1. `get_info(file_ndvi)`
This function extracts the geo-transform, projection, and data type information from the input NDVI raster file.


In [ ]:
# Extract geo-transform, projection, and data type from NDVI file
def get_info(file_ndvi):
    ndvi_tif = gdal.Open(file_ndvi, gdal.GA_ReadOnly)
    ndvi_band = ndvi_tif.GetRasterBand(1)
    gt = ndvi_tif.GetGeoTransform()
    proj = ndvi_tif.GetProjectionRef()
    dtype = ndvi_band.DataType
    return gt, proj, dtype


### 2. `fit(MiniList, MaxList)`
Fits polynomial regression lines for the wet and dry edges based on the minimum and maximum LST values.


In [ ]:
# Fit polynomial regression for wet (MinPfit) and dry (MaxPfit) edges
def fit(MiniList, MaxList):
    ndvi_vector = np.round(np.arange(0.01, 1.01, 0.01), 2)
    MiniList_fin = []
    ndvi_fin = []
    
    for i, val in enumerate(MiniList):
        if np.isfinite(val):
            MiniList_fin.append(val)
            ndvi_fin.append(ndvi_vector[i])
    
    MinPfit = np.polyfit(ndvi_fin[19:90], MiniList_fin[19:90], 1)
    
    MaxList_fin = []
    ndvi_fin = []
    for i, val in enumerate(MaxList):
        if np.isfinite(val):
            MaxList_fin.append(val)
            ndvi_fin.append(ndvi_vector[i])
    
    MaxPfit = np.polyfit(ndvi_fin[19:90], MaxList_fin[19:90], 1)
    
    return MinPfit, MaxPfit


### 3. `compute_tvdi(ndvi, lst, MinPfit, MaxPfit)`
This function calculates the TVDI based on the NDVI, LST, and the polynomial fits for dry and wet edges.


In [ ]:
# Calculate TVDI
def compute_tvdi(ndvi, lst, MinPfit, MaxPfit):
    a1, b1 = MaxPfit
    a2, b2 = MinPfit

    Ts_max = b1 + (a1 * ndvi)
    Ts_min = b2 + (a2 * ndvi)

    TVDI = (lst - Ts_min) / (Ts_max - Ts_min)
    
    return TVDI


### 4. `plot_scatter(ndvi, lst, MiniList, MaxList, MinPfit, MaxPfit, date, scatter_file=None)`
Generates a scatter plot of NDVI vs. LST, along with the wet and dry edges.


In [ ]:
# Plot scatter plot of NDVI vs LST
def plot_scatter(ndvi, lst, MiniList, MaxList, MinPfit, MaxPfit, date, scatter_file=None):
    ndvi_vector = np.round(np.arange(0.01, 1.01, 0.01), 2)
    a1, b1 = MaxPfit
    a2, b2 = MinPfit
    linhamax = [b1 + (a1 * 0), b1 + (a1 * 1)]
    linhamin = [b2 + (a2 * 0), b2 + (a2 * 1)]

    plt.plot(ndvi.ravel(), lst.ravel(), "+", color='dimgray', markersize=4)
    plt.plot(ndvi_vector[14:89], MiniList[14:89], '+', color='b')
    plt.plot(ndvi_vector[14:89], MaxList[14:89], '+', color='r')
    
    if a1 > 0:
        plt.plot([0, 1], linhamax, color='r', markersize=8, label=f"Tsmax = {'%.1f'% b1} + {'%.1f' % abs(a1)} * ndvi")
    else:
        plt.plot([0, 1], linhamax, color='r', markersize=8, label=f"Tsmax = {'%.1f'% b1} - {'%.1f' % abs(a1)} * ndvi")
    
    if a2 > 0:
        plt.plot([0, 1], linhamin, color='b', markersize=8, label=f"Tsmin = {'%.1f' % b2} + {'%.1f' % abs(a2)} * ndvi")
    else:
        plt.plot([0, 1], linhamin, color='b', markersize=8, label=f"Tsmin = {'%.1f' % b2} - {'%.1f' % abs(a2)} * ndvi")
    
    plt.legend(loc='upper right', fontsize=12)
    plt.ylim(top=340, bottom=270)
    plt.xlabel("ndvi")
    plt.ylabel("lst (K)")
    plt.title(f"ndvi vs lst Scatterplot ({date})")
    
    if scatter_file is not None:
        plt.savefig(scatter_file)
    
    plt.show()


### 5. `get_min_max(ndvi, lst)`
This function calculates the minimum and maximum LST values for each NDVI value.


In [ ]:
# Get min and max LST for each NDVI
def get_min_max(ndvi, lst):
    MiniList = []
    MaxList = []
    ndvi_vector = np.round(np.arange(0.01, 1.01, 0.01), 2)

    for val in ndvi_vector:
        indices = np.where((ndvi >= val - 0.001) & (ndvi <= val + 0.001))[0]
        
        if len(indices) > 0:
            lst_min_val = np.nanmin(lst[indices])
            lst_max_val = np.nanmax(lst[indices])
        else:
            lst_min_val = np.nan
            lst_max_val = np.nan
        
        MiniList.append(lst_min_val)
        MaxList.append(lst_max_val)

    return MiniList, MaxList


### 6. `show_tvdi(tvdi, fig_file=None)`
Displays the computed TVDI using a heatmap.


In [ ]:
# Display TVDI result
def show_tvdi(tvdi, fig_file=None):
    plt.imshow(tvdi, cmap='jet_r', vmax=1, vmin=0)
    plt.axis('off')
    plt.colorbar()
    plt.title("TVDI")
    
    if fig_file is not None:
        plt.savefig(fig_file)
    
    plt.show()


### 7. `save_tvdi(TVDI, gt, proj, dtype, file_out, lst)`
Saves the computed TVDI as a GeoTIFF file.


In [ ]:
# Save TVDI result as GeoTIFF
def save_tvdi(TVDI, gt, proj, dtype, file_out, lst):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = lst.shape
    
    if len(TVDI.shape) == 1:
        TVDI = TVDI.reshape(rows, cols)
    
    dset_output = driver.Create(file_out, cols, rows, 1, gdal.GDT_Float32)
    dset_output.SetGeoTransform(gt)
    dset_output.SetProjection(proj)
    dset_output.GetRasterBand(1).WriteArray(TVDI)
    dset_output = None


### 8. `get_data(ndvi_path, lst_path)`
Reads the NDVI and LST raster files.


In [ ]:
# Load NDVI and LST data from files
def get_data(ndvi_path, lst_path):
    ndvi_dataset = gdal.Open(ndvi_path)
    lst_dataset = gdal.Open(lst_path)

    ndvi = ndvi_dataset.ReadAsArray().astype(float)
    lst = lst_dataset.ReadAsArray().astype(float)

    return ndvi, lst


## Main Processing Loop
The following loop processes each NDVI file, finds the corresponding LST file, computes the TVDI, and saves the result.


In [ ]:
# Directory paths and multi-factor to scale NDVI
ndvi_files = glob('path_to_ndvi_files/*.tif')  # Replace with your NDVI file path
lst_dir = 'path_to_lst_files/'  # Replace with your LST file path
output_dir = 'path_to_save_results/'  # Replace with your desired output path

multi = 0.0001  # Scaling factor for NDVI data

# Loop through each NDVI file and process
for ndvi_file in ndvi_files:
    base_filename = os.path.basename(ndvi_file)
    date = base_filename.split("_")[2].split(".")[0]
    lst_file = lst_dir + f'LST_{date}.tif'  # Adjust naming convention accordingly

    # Load NDVI and LST data
    ndvi, lst = get_data(ndvi_file, lst_file)

    # Rescale NDVI
    ndvi *= multi

    # Calculate min and max LST values
    MiniList, MaxList = get_min_max(ndvi, lst)

    # Fit polynomial regression lines
    MinPfit, MaxPfit = fit(MiniList, MaxList)

    # Calculate TVDI
    TVDI = compute_tvdi(ndvi, lst, MinPfit, MaxPfit)

    # Save TVDI result
    file_out = output_dir + f'TVDI_{date}.tif'
    gt, proj, dtype = get_info(ndvi_file)
    save_tvdi(TVDI, gt, proj, dtype, file_out, lst)

    # Plot scatter plot
    plot_scatter(ndvi, lst, MiniList, MaxList, MinPfit, MaxPfit, date)

    # Display TVDI heatmap
    show_tvdi(TVDI)


This notebook processes NDVI and LST data to calculate and visualize the TVDI for monitoring drought conditions, saving results as GeoTIFF files.